In [5]:
import numpy as np
from scipy import stats
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.io import savemat
from scipy.io import loadmat
from numpy import genfromtxt
from sklearn.linear_model import LogisticRegression
from DAPCA import DAPCA
import sklearn.metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import scipy.io
import scipy.linalg
from timeit import default_timer as timer
from datetime import timedelta
import random
from scipy.io import loadmat
from sklearn.svm import SVC
import random
from random import sample
from warnings import filterwarnings
filterwarnings('ignore')

def space_alignment(X,Y,Subspace_Dim,normalize=True):
    if normalize:
        Xz = stats.zscore((X.T/np.sum(X,axis=1)).T)
        Yz = stats.zscore((Y.T/np.sum(Y,axis=1)).T)
    else:
        Xz = X
        Yz = Y
    pcaX = PCA()
    pcaX.fit(Xz)
    VX = pcaX.components_.T
    pcaY = PCA()
    pcaY.fit(Yz)
    VY = pcaY.components_.T
    if Subspace_Dim>VX.shape[1]:
        Subspace_Dim = VX.shape[1]
    if Subspace_Dim>VY.shape[1]:
        Subspace_Dim = VY.shape[1]
    VX = VX[:,:Subspace_Dim]
    VY = VY[:,:Subspace_Dim]
    Xpa = Xz@(VX@VX.T@VY)
    Ypa = Yz@VY
    return [Xpa,Ypa]

def CORAL(Xs,Xt,eps=1e-10):
    #Xw, _ = whiten(Xs,eps=eps)
    cov_source = np.cov(Xs.T) + eps*np.eye(Xs.shape[1])
    cov_target = np.cov(Xt.T) + eps*np.eye(Xt.shape[1])
    Xs_coral = np.real(Xs@np.linalg.inv(scipy.linalg.sqrtm(cov_source))@scipy.linalg.sqrtm(cov_target))
    return Xs_coral

def print_accuracies(X,labels,Y,target_labels,method_name):
    clf = LogisticRegression(random_state=42,solver='lbfgs',multi_class='multinomial',max_iter=1000) 
    clf.fit(X, labels)
    y_pred = clf.predict(Y)
    acc = sklearn.metrics.accuracy_score(target_labels, y_pred)
    print('Accuracy of '+method_name+'(Logistic Regression)=',acc)
    acc = sklearn.metrics.balanced_accuracy_score(target_labels, y_pred)
    print('Balanced accuracy of '+method_name+'(Logistic Regression)=', acc)    

In [6]:
# DVD / Books
X = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_train.mat')['data']
labels = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_train.mat')['labels']
labels = labels[0].T
Y = scipy.io.loadmat('datasets/Amazon_Reviews/books_test.mat')['data']
target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/books_test.mat')['labels']
target_labels = target_labels[0].T

# # Books / DVD
# X = scipy.io.loadmat('datasets/Amazon_Reviews/books_train.mat')['data']
# labels = scipy.io.loadmat('datasets/Amazon_Reviews/books_train.mat')['labels']
# Y = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_test.mat')['data']
# target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_test.mat')['labels']

# # DVD / Electronics
# X = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_train.mat')['data']
# labels = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_train.mat')['labels']
# Y = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_test.mat')['data']
# target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_test.mat')['labels']

# # Books / Kitchen
# X = scipy.io.loadmat('datasets/Amazon_Reviews/books_train.mat')['data']
# labels = scipy.io.loadmat('datasets/Amazon_Reviews/books_train.mat')['labels']
# Y = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_test.mat')['data']
# target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_test.mat')['labels']

# # DVD / Kitchen
# X = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_train.mat')['data']
# labels = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_train.mat')['labels']
# Y = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_test.mat')['data']
# target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_test.mat')['labels']

# # Kitchen / Books
# X = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_train.mat')['data']
# labels = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_train.mat')['labels']
# Y = scipy.io.loadmat('datasets/Amazon_Reviews/books_test.mat')['data']
# target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/books_test.mat')['labels']

# # Books / Electronics
# X = scipy.io.loadmat('datasets/Amazon_Reviews/books_train.mat')['data']
# labels = scipy.io.loadmat('datasets/Amazon_Reviews/books_train.mat')['labels']
# Y = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_test.mat')['data']
# target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_test.mat')['labels']

# # Electronics / DVD
# X = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_train.mat')['data']
# labels = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_train.mat')['labels']
# Y = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_test.mat')['data']
# target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_test.mat')['labels']

# # Kitchen / Electronics
# X = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_train.mat')['data']
# labels = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_train.mat')['labels']
# Y = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_test.mat')['data']
# target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_test.mat')['labels']

# # Kitchen / DVD
# X = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_train.mat')['data']
# labels = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_train.mat')['labels']
# Y = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_test.mat')['data']
# target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/dvd_test.mat')['labels']

# # Electronics / Books
# X = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_train.mat')['data']
# labels = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_train.mat')['labels']
# Y = scipy.io.loadmat('datasets/Amazon_Reviews/books_test.mat')['data']
# target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/books_test.mat')['labels']

# # Electronics / Kitchen
# X = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_train.mat')['data']
# labels = scipy.io.loadmat('datasets/Amazon_Reviews/electronics_train.mat')['labels']
# Y = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_test.mat')['data']
# target_labels = scipy.io.loadmat('datasets/Amazon_Reviews/kitchen_test.mat')['labels']

In [7]:
point_size = 10

ind1 = np.where(labels== -1)[0]
ind2 = np.where(labels== 1)[0]
target_ind1 = np.where(target_labels== -1)[0]
target_ind2 = np.where(target_labels== 1)[0]

#Xn = stats.zscore(X)
#Yn = stats.zscore(Y)
Xn = X-np.mean(X,axis=0)
Yn = Y-np.mean(Y,axis=0)

pca = PCA(n_components=200)
Xp = pca.fit_transform(np.concatenate([Xn,Yn]))
XpX = Xp[:X.shape[0],:]
XpY = Xp[X.shape[0]:,:]

print_accuracies(XpX,labels,XpY,target_labels,'PCA')

# plt.figure(figsize=(8,8))
# plt.scatter(XpX[ind1,0],XpX[ind1,1],c='g',s=point_size)
# plt.scatter(XpX[ind2,0],XpX[ind2,1],c='y',s=point_size)
# plt.scatter(XpY[target_ind1,0],XpY[target_ind1,1],c='r',s=point_size)
# plt.scatter(XpY[target_ind2,0],XpY[target_ind2,1],c='b',s=point_size)
# #plt.title('PCA before space alignment, raw data',fontsize=20)
# plt.title('PCA, raw data',fontsize=20)
# plt.xlabel('PCA1',fontsize=20)
# plt.ylabel('PCA2',fontsize=20)
# plt.show()

start = timer()
[Xpa,Ypa] = space_alignment(Xn,Yn,200,normalize=False)
end = timer()
print('Time to compute SA:',timedelta(seconds=end-start))

print_accuracies(Xpa,labels,Ypa,target_labels,'SA')

# plt.figure(figsize=(8,8))
# plt.scatter(Xpa[ind1,0],Xpa[ind1,1],c='g',s=point_size)
# plt.scatter(Xpa[ind2,0],Xpa[ind2,1],c='y',s=point_size)
# plt.scatter(Ypa[target_ind1,0],Ypa[target_ind1,1],c='r',s=point_size)
# plt.scatter(Ypa[target_ind2,0],Ypa[target_ind2,1],c='b',s=point_size)
# plt.xlabel('SA1',fontsize=20)
# plt.ylabel('SA2',fontsize=20)
# #plt.title('Space Alignment, acc={:2.2f}'.format(acc),fontsize=20)
# plt.title('Space Alignment (SA)',fontsize=20)
# #plt.axis('equal')
# plt.show()

Accuracy of PCA(Logistic Regression)= 0.683090705487122
Balanced accuracy of PCA(Logistic Regression)= 0.6819118718924742
Time to compute SA: 0:00:34.490950
Accuracy of SA(Logistic Regression)= 0.6600223964165733
Balanced accuracy of SA(Logistic Regression)= 0.659170843481039


In [8]:
point_size = 10

ind1 = np.where(labels== -1)[0]
ind2 = np.where(labels== 1)[0]
target_ind1 = np.where(target_labels== -1)[0]
target_ind2 = np.where(target_labels== 1)[0]

# ======================= PARAMETERS OF DAPCA ===========================
alpha = 0
gamma = 1
maxIter = 10
beta = 1
kNN = 5
num_comps = 200
# =======================================================================

pca = PCA(svd_solver='full')
u = pca.fit_transform(X)
mn = np.mean(X,axis=0)
PY = pca.fit_transform(Y)

start = timer()
[V1, D1, PX, PY, kNNs] = DAPCA(X, labels, num_comps,  YY=Y, 
                         alpha=alpha, gamma=gamma,maxIter=maxIter, 
                        beta=beta,verbose='all',kNN=kNN,
                              eps=1e-10) 

end = timer()
print('Time to compute DAPCA:',timedelta(seconds=end-start))

print_accuracies(PX,labels,PY,target_labels,'DAPCA')

# plt.figure(figsize=(8,8))
# plt.scatter(PX[ind2,0],PX[ind2,1],c='y',s=point_size)
# plt.scatter(PX[ind1,0],PX[ind1,1],c='g',s=point_size)
# plt.scatter(PY[target_ind1,0],PY[target_ind1,1],c='r',s=point_size)
# plt.scatter(PY[target_ind2,0],PY[target_ind2,1],c='b',s=point_size)
# plt.xlabel('DAPCA1',fontsize=20)
# plt.ylabel('DAPCA2',fontsize=20)
# #plt.title('Domain adaptation PCA, acc={:2.2f}'.format(acc),fontsize=20)
# plt.title('Domain adaptation PCA (DAPCA)',fontsize=20)
# plt.show()

### NOW DAPCA in the mode of SUPERVISED TCA

[V1, D1, PXtca, PYtca, kNNs] = DAPCA(Xn, labels, num_comps,  YY=Yn, 
                         alpha=alpha, gamma=gamma,maxIter=maxIter, 
                        beta=beta,verbose='all',kNN=kNN,tca=1,
                              eps=1e-10) 

end = timer()
print('Time to compute STCA:',timedelta(seconds=end-start))

print_accuracies(PXtca,labels,PYtca,target_labels,'STCA')

# plt.figure(figsize=(8,8))
# plt.scatter(PXtca[ind2,0],PXtca[ind2,1],c='y',s=point_size)
# plt.scatter(PXtca[ind1,0],PXtca[ind1,1],c='g',s=point_size)
# plt.scatter(PYtca[target_ind1,0],PYtca[target_ind1,1],c='r',s=point_size)
# plt.scatter(PYtca[target_ind2,0],PYtca[target_ind2,1],c='b',s=point_size)
# plt.xlabel('STCA1',fontsize=20)
# plt.ylabel('STCA2',fontsize=20)
# #plt.xlim([-5,5])
# #plt.ylim([-5,5])
# #plt.axis('equal')
# plt.title('Supervised TCA',fontsize=20)
# plt.show()

Iteration: 0 non-neg: 200 Hw: 283.72626542729296
Iteration: 1 non-neg: 200 Hw diff: 0.020876939084403415 (289.7758992235104)
Iteration: 2 non-neg: 200 Hw diff: 0.0018783680084461158 (290.321229332862)
Iteration: 3 non-neg: 200 Hw diff: 0.0006973987617297148 (290.5238402993397)
Iteration: 4 non-neg: 200 Hw diff: 0.00028664462434662433 (290.6071412741827)
Iteration: 5 non-neg: 200 Hw diff: 0.00020276931285404329 (290.6660794353798)
Iteration: 6 non-neg: 200 Hw diff: 0.0001804550733480296 (290.7185410710324)
Iteration: 7 non-neg: 200 Hw diff: 8.671622318875787e-05 (290.7437532712318)
Iteration: 8 non-neg: 200 Hw diff: 3.751536211777661e-05 (290.75466103762807)
Iteration: 9 non-neg: 200 Hw diff: 1.8480275272891988e-05 (290.76003436310145)
Time to compute DAPCA: 0:08:51.340024
Accuracy of DAPCA(Logistic Regression)= 0.7368421052631579
Balanced accuracy of DAPCA(Logistic Regression)= 0.7369003287936899
Iteration: 0 non-neg: 200 Hw: 376.13231900738856
Time to compute STCA: 0:09:36.628426
Accu

In [9]:
ind1 = np.where(labels== -1)[0]
ind2 = np.where(labels== 1)[0]
target_ind1 = np.where(target_labels== -1)[0]
target_ind2 = np.where(target_labels== 1)[0]

#Xn = stats.zscore(X)
#Yn = stats.zscore(Y)
Xn = X-np.mean(X,axis=0)
Yn = Y-np.mean(Y,axis=0)

print_accuracies(Xn,labels,Yn,target_labels,'Baseline')

pca = PCA(n_components=200)
Xp = pca.fit_transform(np.concatenate([Xn,Yn]))
XpX = Xp[:X.shape[0],:]
XpY = Xp[X.shape[0]:,:]

print_accuracies(XpX,labels,XpY,target_labels,'PCA')

# plt.figure(figsize=(8,8))
# plt.scatter(XpY[target_ind1,0],XpY[target_ind1,1],c='r',s=point_size)
# plt.scatter(XpY[target_ind2,0],XpY[target_ind2,1],c='b',s=point_size)
# plt.scatter(XpX[ind2,0],XpX[ind2,1],c='y',s=point_size)
# plt.scatter(XpX[ind1,0],XpX[ind1,1],c='g',s=point_size)
# plt.xlabel('PCA1',fontsize=20)
# plt.ylabel('PCA2',fontsize=20)
# #plt.xlim([-5,5])
# #plt.ylim([-5,5])
# plt.title('PCA',fontsize=20)
# #plt.axis('equal')
# plt.show()

Xn = stats.zscore(X)
Yn = stats.zscore(Y)
X_coral = CORAL(Xn,Yn,eps=1)
# This is CORAL computed using MATLAB (done for making sure Python code gives the same result)
# X_coral = np.loadtxt('2clusters_3d_X_coral.csv',delimiter=',')

# plt.figure(figsize=(8,8))
# plt.scatter(Yn[target_ind1,0],Yn[target_ind1,1],c='r',s=point_size)
# plt.scatter(Yn[target_ind2,0],Yn[target_ind2,1],c='b',s=point_size)
# plt.scatter(X_coral[ind2,0],X_coral[ind2,1],c='y',s=point_size)
# plt.scatter(X_coral[ind1,0],X_coral[ind1,1],c='g',s=point_size)
# plt.xlabel('x1',fontsize=40)
# plt.ylabel('x2',fontsize=40)
# #plt.xlim([-5,5])
# #plt.ylim([-5,5])
# #plt.title('CORAL, acc={:2.2f}'.format(acc),fontsize=20)
# plt.title('CORAL',fontsize=40)
# #plt.axis('equal')
# plt.show()

# plt.figure(figsize=(8,8))
# plt.scatter(Yn[target_ind1,0],Yn[target_ind1,2],c='r',s=point_size)
# plt.scatter(Yn[target_ind2,0],Yn[target_ind2,2],c='b',s=point_size)
# plt.scatter(X_coral[ind2,0],X_coral[ind2,2],c='y',s=point_size)
# plt.scatter(X_coral[ind1,0],X_coral[ind1,2],c='g',s=point_size)
# plt.xlabel('x1',fontsize=40)
# plt.ylabel('x3',fontsize=40)
# #plt.xlim([-5,5])
# #plt.ylim([-5,5])
# #plt.title('CORAL, acc={:2.2f}'.format(acc),fontsize=20)
# plt.title('CORAL',fontsize=40)
# #plt.axis('equal')
# plt.show()

# plt.figure(figsize=(8,8))
# plt.scatter(Yn[target_ind1,1],Yn[target_ind1,2],c='r',s=point_size)
# plt.scatter(Yn[target_ind2,1],Yn[target_ind2,2],c='b',s=point_size)
# plt.scatter(X_coral[ind2,1],X_coral[ind2,2],c='y',s=point_size)
# plt.scatter(X_coral[ind1,1],X_coral[ind1,2],c='g',s=point_size)
# plt.xlabel('x2',fontsize=40)
# plt.ylabel('x3',fontsize=40)
# #plt.xlim([-5,5])
# #plt.ylim([-5,5])
# #plt.title('CORAL, acc={:2.2f}'.format(acc),fontsize=20)
# plt.title('CORAL',fontsize=40)
# #plt.axis('equal')
# plt.show()

print_accuracies(X_coral,labels,Yn,target_labels,'Full CORAL')

pca = PCA(n_components=200)
Xp = pca.fit_transform(np.concatenate([X_coral,Yn]))
XpX = Xp[:X.shape[0],:]
XpY = Xp[X.shape[0]:,:]

print_accuracies(XpX,labels,XpY,target_labels,'PCA CORAL')

# plt.figure(figsize=(8,8))
# plt.scatter(XpY[target_ind1,0],XpY[target_ind1,1],c='r',s=point_size)
# plt.scatter(XpY[target_ind2,0],XpY[target_ind2,1],c='b',s=point_size)
# plt.scatter(XpX[ind2,0],XpX[ind2,1],c='y',s=point_size)
# plt.scatter(XpX[ind1,0],XpX[ind1,1],c='g',s=point_size)
# plt.xlabel('CORAL PCA1',fontsize=40)
# plt.ylabel('CORAL PCA2',fontsize=40)
# #plt.xlim([-5,5])
# #plt.ylim([-5,5])
# #plt.title('CORAL, acc={:2.2f}'.format(acc),fontsize=20)
# plt.title('CORAL',fontsize=40)
# #plt.axis('equal')
# plt.show()

Accuracy of Baseline(Logistic Regression)= 0.7379619260918253
Balanced accuracy of Baseline(Logistic Regression)= 0.7373977135352867
Accuracy of PCA(Logistic Regression)= 0.6884658454647257
Balanced accuracy of PCA(Logistic Regression)= 0.6873449748989778


ValueError: array must not contain infs or NaNs